In [2]:
# import pyspark
# spark = pyspark.sql.SparkSession.builder\
#     .master('local[*]')\
#     .appName('Shengwei\'s first spark notebook')\
#     .getOrCreate()

In [3]:
# df = spark.read.format("csv").option("header","true").load("accidents_2005_to_2007.csv")

In [5]:
import folium
from folium import plugins
import branca
import pandas as pd

In [6]:
# TODO load data from mongodb
df = pd.read_csv("accidents_2005_to_2007.csv")

/opt/conda/envs/bigdata/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,1,2,1,1,04/01/2005,...,Zebra crossing,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,1,Yes,E01002849,2005
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,1,3,1,1,05/01/2005,...,Pedestrian phase at traffic signal junction,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01002909,2005
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,1,3,2,1,06/01/2005,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01002857,2005
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,1,3,1,1,07/01/2005,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01002840,2005
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,1,3,1,1,10/01/2005,...,No physical crossing within 50 meters,Darkness: Street lighting unknown,Fine without high winds,Wet/Damp,None,None,1,Yes,E01002863,2005


In [8]:
def ping_map(df, LSOA_of_Accident_Location, Year, zoom_start):
    df_filtered = df[(df["LSOA_of_Accident_Location"]==LSOA_of_Accident_Location)&(df["Year"]==Year)]
    location = (df_filtered["Latitude"].mean(),df_filtered["Longitude"].mean())
    uk_map = folium.Map(location=location,zoom_start=zoom_start)
    title_color = "#B0E0E6"
    value_color = "#F0F8FF"
    for index, row in df_filtered.iterrows():
        if row.Accident_Severity == 1:
            color = 'blue'
        elif row.Accident_Severity == 2:
            color = 'green'
        else:
            color = 'red'
        html_page = f"""<!DOCTYPE html>
                        <html>

                        <head>
                            <h2>Detailed Infomation of accident {row.Accident_Index}</h4>
                        </head>
                        <table style="height: 130px; width: 600px;">
                        <tbody>
                        <tr>
                        <td style="background-color: {title_color};"><div style="text-align: center;">Date</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Date}</td>
                        </tr>
                        <tr>
                        <td style="background-color: {title_color};"><div style="text-align: center;">Number of Vehicles</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Number_of_Vehicles}</td>
                        </tr>

                        <tr>
                        <td style="background-color: {title_color};"><div style="text-align: center;">Number of Casualties</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Number_of_Casualties}</td>
                        </tr>

                        <tr>
                        <td style="background-color: {title_color};"><div style="text-align: center;">Time</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Time}</td>
                        </tr>

                        <tr>
                        <td style="background-color: {title_color};"><div style="text-align: center;">Light Conditions</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Light_Conditions}</td>
                        </tr>

                        <tr>
                        <td style="background-color: {title_color};"><div style="text-align: center;">Weather Conditions</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Weather_Conditions}</td>
                        </tr>

                        <tr>
                        <td style="background-color: {title_color};width: 300px"><div style="text-align: center;">Road Surface Conditions</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Road_Surface_Conditions}</td>
                        </tr>

                        <tr>
                        <td style="background-color: {title_color};width: 300px"><div style="text-align: center;">Road Type</div></td>
                        <td style="width: 300px;background-color: {value_color};">{row.Road_Type}</td>
                        </tr>

                        </tbody>
                        </table>
                        </html>
                        """
        iframe = branca.element.IFrame(html=html_page,width=600,height=300)
        popup = folium.Popup(iframe, parse_html=True) 
        folium.Marker([row.Latitude, row.Longitude],popup=popup, icon=folium.Icon(color=color, icon="remove-sign")).add_to(uk_map)
    return uk_map

In [21]:
LSOA_of_Accident_Location = "E01002849"
Year = 2005
zoom_start = 15
uk_map = ping_map(df,LSOA_of_Accident_Location,Year,zoom_start)

In [22]:
uk_map

In [20]:
def hitmap(df, LSOA_of_Accident_Location, Year, zoom_start, target_col):
    df_filtered = df[(df["LSOA_of_Accident_Location"]==LSOA_of_Accident_Location)&(df["Year"]==Year)]
    location = (df_filtered["Latitude"].mean(),df_filtered["Longitude"].mean())
    print(location)
    uk_map = folium.Map(location=location, zoom_start=zoom_start)
    
    data_heat = df_filtered[['Latitude','Longitude',target_col]].dropna().values.tolist()
    plugins.HeatMap(data_heat).add_to(uk_map)
    return uk_map

In [19]:
LSOA_of_Accident_Location = "E01002849"
Year = 2005
zoom_start = 15
target_col = "Number_of_Casualties"
uk_map = hitmap(df, LSOA_of_Accident_Location, Year, zoom_start, target_col)
uk_map

(51.490235, -0.19182911111111112)
